# 05_크림 나이키 데이터

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
# ChromeDriver 설치 및 경로 설정
service = Service(ChromeDriverManager().install())

In [3]:
# 옵션 설정
options = Options()
options.add_experimental_option("detach", True)  # 브라우저 자동 종료 방지
options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 로그 제거

In [5]:
# 페이지 오픈
driver = webdriver.Chrome(service=service, options=options)
url = 'https://kream.co.kr/search?keyword=%EB%82%98%EC%9D%B4%ED%82%A4&tab=products&footer=home'
driver.get(url)
time.sleep(3)

In [7]:
# 스크롤
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 제품 요소 수집
items = driver.find_elements(By.CLASS_NAME, 'item_inner')
print(f"총 수집된 제품 개수: {len(items)}개")

# 데이터 파싱
product_data = []
for idx, item in enumerate(items, 1):
    try:
        link = item.get_attribute("href")
        brand = item.find_element(By.CLASS_NAME, 'brand-name').text
        eng_name = item.find_element(By.CLASS_NAME, 'name').text
        kor_name = item.find_element(By.CLASS_NAME, 'translated_name').text
        price = item.find_element(By.CLASS_NAME, 'amount').text
        info_texts = item.find_elements(By.CLASS_NAME, 'wish_figure')
        wish_text = info_texts[0].text if info_texts else ''
        review_text = item.find_element(By.CLASS_NAME, 'review_figure').text

        product_data.append({
            '브랜드': brand,
            '상품명(영문)': eng_name,
            '상품명(한글)': kor_name,
            '가격': price,
            '관심수': wish_text,
            '리뷰수': review_text,
            '상세페이지 링크': link
        })

        print(f"[{idx}] {kor_name} - {price}")
    except Exception as e:
        print(f"[{idx}] 오류 발생: {e}")

총 수집된 제품 개수: 50개
[1] 나이키 에어포스 1 '07 로우 화이트 - 99,000원
[2] 나이키 에어포스 1 '07 WB 플랙스 - 120,000원
[3] 나이키 에어포스 1 '07 로우 트리플 블랙 - 104,000원
[4] (W) 나이키 에어 슈퍼플라이 메탈릭 실버 앤 블랙 - 116,000원
[5] 나이키 줌 플라이 6 화이트 스모크 그레이 - 138,000원
[6] 나이키 줌 플라이 6 화이트 퓨어 플래티넘 - 140,000원
[7] (W) 나이키 스포츠웨어 칠 니트 그래픽 티셔츠 미드나잇 네이비 - 66,000원
[8] (W) 나이키 에어포스 1 '07 WB 플랙스 위트 - 100,000원
[9] 나이키 브라질리아 9.5 트레이닝 더플백 스몰 41L 블랙 - 31,000원
[10] 나이키 리액트X 리주버네이트 슬라이드 블랙 - 61,000원
[11] 나이키 토탈 90 3 SP 라이트 오어우드 브라운 블랙 - 164,000원
[12] 나이키 호나우지뉴 브라질 2004 토탈 90 리이슈 사커 레플리카 저지 (마킹 버전) - 198,000원
[13] 나이키 리액트X 리주버네이트 라이트 오어우드 브라운 - 107,000원
[14] (W) 나이키 에어맥스 뮤즈 블랙 앤 메탈릭 실버 - 151,000원
[15] 나이키 코비 6 프로트로 토탈 오렌지 - 293,000원
[16] 나이키 FC 바르셀로나 2025/26 스타디움 홈 드라이핏 풋볼 레플리카 저지 딥 로얄 블루 노블 레드 (논 마킹 버전) - 121,000원
[17] 나이키 V2K 런 서밋 화이트 메탈릭 실버 - 77,000원
[18] 나이키 줌 플라이 6 블랙 라이트 스모크 그레이 - 119,000원
[19] 나이키 ACG 트레일 캡 블랙 - 43,000원
[20] 나이키 에어 줌 알파플라이 넥스트% 3 화이트 퓨어 플래티넘 - 258,000원
[21] (W) 나이키 에어포스 1 '07 로우 화이트 - 94,000원
[22] 나이키 V2K 런 블랙 앤트러사이트 - 81,000원
[23] 나이

In [8]:
df = pd.DataFrame(product_data)
df.to_csv('data/크림_나이키.csv', index=False, encoding='utf-8-sig')

In [9]:
# 종료
driver.quit()